In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.impute import KNNImputer
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import StratifiedKFold, KFold
from lightgbm import LGBMRegressor, LGBMClassifier

from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import optuna
from optuna.trial import TrialState

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("../input/tabular-playground-series-sep-2021/train.csv")
test_df = pd.read_csv("../input/tabular-playground-series-sep-2021/test.csv")

train_df["nan_count"] = train_df.isnull().sum(axis=1)
test_df["nan_count"] = test_df.isnull().sum(axis=1)

feature_cols = [col for col in train_df.columns if col not in ['id', 'claim']]

X, y = train_df[feature_cols], train_df['claim']
test = test_df[feature_cols]

In [ ]:
train_df.head()

# **Data Transformation**
First step is to impute missing values and then scale the features as the features have very different ranges.

In [ ]:
data_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
    ('scaler', StandardScaler())
])

# **Pipeline - XGB**
The hyperparameters were tuned separately with optuna.

In [ ]:
params_xgb = {'max_depth': 3,
              'n_estimators': 7781,
              'min_child_weight': 7, 
             'learning_rate': 0.051757426392409905, 
             'gamma': 9.213719370277009e-06, 
             'lambda': 0.007979791084907156, 
             'alpha': 0.030419732577886008, 
             'subsample': 0.6768773088432918, 
             'colsample_bytree': 0.6616897821787362, 
             'grow_policy': 'lossguide',
             'random_state': 41,
             'verbosity' : 0}

if  torch.cuda.is_available():
    _device = 'gpu'
else:
    _device = 'cpu'
    
if _device == 'cpu':
    params_xgb['tree_method'] = 'hist'
    params_xgb['prediction'] = 'cpu_predictior'
else:
    params_xgb['tree_method'] = 'gpu_hist'
    params_xgb['prediction'] = 'gpu_predictior'
    
model_xgb = XGBRegressor(**params_xgb)

pipeline_xgb = Pipeline(steps=[
    ('transformer', data_transformer),
    ('model', model_xgb)
])

This Cell uses the whole training data to train the XGB model and makes predictions. This is commented out at the moment as the purpose of this pipeline is to create a pipeline starting from scratch and ending with blending.

In [ ]:
# pipeline_xgb.fit(X, y)
# predictions = pipeline_xgb.predict(test)
# print(predictions[:10])

# **Pipeline - LGBM**

In [ ]:
if  torch.cuda.is_available():
    _device = 'gpu'
else:
    _device = 'cpu'
params_lgbm = {'learning_rate': 0.09497708539776868,
          'n_estimators': 3168,
          'max_depth': 2,
          'reg_alpha': 0.015322467962858254,
          'reg_lambda': 1.7933044558463738e-06,
          'device' : _device,
          'random_state': 41,
          'verbosity' : 0}

model_lgbm = LGBMClassifier(**params_lgbm)

pipeline_lgbm = Pipeline(steps=[
    ('transformer', data_transformer),
    ('model', model_lgbm)
])

In [ ]:
# pipeline_lgbm.fit(X, y)
# predictions = pipeline_lgbm.predict(test)
# print(predictions[:10])

# **K-fold or Crossvlidation Function**
This function uses crossvalidation to create predictions for all training and test data to blenda afterwards.

In [ ]:
def cross_validation_predict(pipe, X, y, test_data, name, do_add=True, n_folds = 10):
    global train_df, test_df
    valid_predictions = {}
    submission = pd.read_csv("../input/tabular-playground-series-sep-2021/sample_solution.csv")
    submission.columns = ['id', 'pred_' + name]
    
    kfold = StratifiedKFold(n_splits = n_folds, random_state=41, shuffle=True)

    predictions = [0.0]*test.shape[0]
    for idx in kfold.split(X=X, y=y):
        train_idx, val_idx = idx[0], idx[1]
        X_train = X.iloc[train_idx]
        y_train = y.iloc[train_idx]
        X_val = X.iloc[val_idx]
        y_val = y.iloc[val_idx]

        pipe.fit(X_train, y_train)

        val_preds = pipe.predict(X_val)
        print(roc_auc_score(y_val, val_preds))
        if do_add:
            valid_predictions.update(dict(zip(val_idx, val_preds)))
        preds = pipe.predict(test_data)
        predictions += preds / kfold.n_splits
        
    if do_add:
        valid_predictions = pd.DataFrame.from_dict(valid_predictions, orient="index").reset_index()
        valid_predictions.columns = ["id", "pred_" + name]
        train_df = train_df.merge(valid_predictions, on="id", how="left")

        submission['pred_' + name] = preds
        test_df = test_df.merge(submission, on="id", how="left")

        
    return predictions

The next four cells create data for blending using the pipelines and crossvalidation function.

In [ ]:
predictions = cross_validation_predict(pipeline_lgbm, X, y, test, "lgbm", n_folds = 10)

print(predictions[:10])

In [ ]:
test_df.head()

In [ ]:
predictions = cross_validation_predict(pipeline_xgb, X, y, test, "xgb", n_folds = 10)

print(predictions[:10])

In [ ]:
train_df.head()

# **Blending function**

In [ ]:
def blender_predict(model, useful_features):
    global train_df, test_df
    X_train = train_df[useful_features]
    y_train = train_df.claim
    test = test_df[useful_features]
    
    model.fit(X_train, y_train)
    predictions = model.predict(test)
    
    return predictions

This cell makes two predictions, one with blending on the full dataset and another with K-fold.

In [ ]:
from sklearn.linear_model import LinearRegression
useful_features = ['pred_xgb', 'pred_lgbm']

predictions = blender_predict(LinearRegression(), useful_features)
print(predictions[:10])

X_new = train_df[useful_features]
y_new = train_df.claim
test_new = test_df[useful_features]
predictions_2 = cross_validation_predict(LinearRegression(), X_new, y_new, test_new, "none", False)
print(predictions_2[:10])

In [ ]:
submission = pd.read_csv("../input/tabular-playground-series-sep-2021/sample_solution.csv")
submission.claim = predictions_2
submission.to_csv("submission.csv", index=False)